In [4]:
import keras
keras.__version__

/Users/chrisheinrich/anaconda/envs/deep/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.5'

# Making predictions of test data

In [5]:
from keras.models import load_model

# Load weights of model trained on AWS
model = load_model('./weights/name_that_whale_700.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 6, 6, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               18874624  
_________________________________________________________________
dense_12 (Dense)             (None, 4251)              1092507   
Total params: 40,828,611
Trainable params: 28,369,491
Non-trainable params: 12,459,120
_________________________________________________________________


**Loading the test images**

In [6]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

home_dir = os.getcwd()
test_dir = os.path.join(home_dir, 'test/')
test_list = os.listdir(test_dir)
test_count = len(test_list)
print(test_count)


15610


In [7]:
import cv2
from keras.preprocessing import image

# Model was trained using images of this size
image_size = (180,180)

def load_image(img_path):
    img = image.load_img(img_path, target_size = image_size)
    x = image.img_to_array(img)
    x /= 255
    return x

In [16]:
import os
import numpy as np

home_dir = os.getcwd()
fname = os.path.join(home_dir, 'targets.csv') # targets for both train and validation

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
lines = lines[:-1]

whale_ids = [line.split(',')[1] for line in lines]
whale_ids = set(whale_ids) # convert to set to remove duplicats
whale_ids = list(whale_ids) # convert back to list to make it ordered
ids_count = len(whale_ids)

print(header)
print(len(lines))
print(ids_count)

['Image', 'Id']
9850
4251


## Predicting labels

In [17]:
batch_size = 10

def predict_labels():
    labels = np.zeros(shape=(test_count,ids_count))
    for i in range(10):
        batch = np.zeros(shape=(batch_size,180,180,3))
        for j in range(batch_size):
            batch[j] = load_image(os.path.join(test_dir,test_list[i*batch_size + j]))
        labels_batch = model.predict(batch)
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    return labels
            
        

In [25]:
label_prediction = predict_labels()

In [26]:
# This function takes in a label vector, and returns the top 5 whale ids in order
def get_ids(label_vec):
    ids = ''
    for i in range(5):
        max_value = max(label_vec)
        max_index = np.where(label_vec==max_value)
        ids += whale_ids[max_index[0][0]]
        label_vec[max_index] = -1
        if i is not 4:
            ids += ' '
    return ids

In [28]:
prediction = 'Image,Id \n'
for i in range(100):
    prediction += test_list[i]
    prediction += ', '
    prediction += get_ids(label_prediction[i])
    prediction += '\n'

    
print(prediction)

Image,Id 
cb2aa96f.jpg,w_1c69443 w_e3bb130 w_2c75202 w_a3cd405 w_a7c6cf3
703b6a1d.jpg,w_dbc392b w_80c692d w_e9ffbe9 w_62799b5 w_0d48a7d
ce79ac9d.jpg,w_80c692d w_4a4546c w_80c692d w_80c692d w_80c692d
7cd1936f.jpg,w_7b5a630 w_046634b w_dac2f44 w_0b3b659 w_80c692d
c1da857c.jpg,w_80c692d w_7b3f9d1 w_2f81b85 w_f0f56dc w_62799b5
1343b5ca.jpg,w_80c692d w_9122e43 w_f70bf4d w_70b9e55 w_3f907d6
4378186a.jpg,w_80c692d w_d7ffaf2 w_0e9f6d9 w_616ca36 w_f9fb9d7
d97bb43b.jpg,w_80c692d w_c01beb7 w_01ab6dc w_4dd966d w_7417b12
6c024503.jpg,w_80c692d w_7417b12 w_9a0c14d w_65efe4d w_c01beb7
e14189c6.jpg,w_57e8a80 w_1ec0481 w_0466071 w_c053b99 w_3e4bb3f
b6188d6c.jpg,w_2f81b85 w_1febbf3 w_80c692d w_3e9d82e w_07274b2
cf2a74d8.jpg,w_61b87f9 w_9a4bd76 w_817f022 w_0bd3671 w_7b5a630
dff4b248.jpg,w_82bab38 w_20c671c w_697c72e w_3eaef21 w_68820e8
265a33a3.jpg,w_a8335f2 w_80c692d w_798bf53 w_56ddb00 w_4a4546c
e3c0cfa0.jpg,w_80c692d w_ba2f2c4 w_2085f2e w_d037229 w_8e3ebeb
6c0c7e05.jpg,w_80c692d w_97e7648 w_56bbc91 w_